In [27]:
import requests
import pandas as pd
import json
import my_api_key as key


In [28]:
api_key = key.api_key

In [29]:
lat= 49.314695
lon= -2.833333
url = "https://api.openweathermap.org/data/3.0/onecall?lat={}&lon={}&appid={}".format(lat,lon,api_key)

In [30]:
response = requests.get(url)
data = response.json()
print(json.dumps(data, indent=4))

{
    "lat": 49.3147,
    "lon": -2.8333,
    "timezone": "Europe/Guernsey",
    "timezone_offset": 3600,
    "current": {
        "dt": 1712227591,
        "sunrise": 1712209286,
        "sunset": 1712256398,
        "temp": 286.78,
        "feels_like": 286.44,
        "pressure": 1008,
        "humidity": 86,
        "dew_point": 284.48,
        "uvi": 3.24,
        "clouds": 62,
        "visibility": 10000,
        "wind_speed": 10.47,
        "wind_deg": 230,
        "wind_gust": 16.1,
        "weather": [
            {
                "id": 803,
                "main": "Clouds",
                "description": "broken clouds",
                "icon": "04d"
            }
        ]
    },
    "minutely": [
        {
            "dt": 1712227620,
            "precipitation": 0
        },
        {
            "dt": 1712227680,
            "precipitation": 0
        },
        {
            "dt": 1712227740,
            "precipitation": 0
        },
        {
            "dt": 1712227

In [31]:
df = pd.read_csv("gps_data.csv")
df.head()


,Ville,Latitude,Longitude
0,Mont Saint Michel,48.635954,-1.511460
1,St Malo,49.314695,-96.953823
2,Bayeux,49.276462,-0.702474
3,Le Havre,49.493898,0.107973
4,Rouen,49.440459,1.093966


In [32]:
# Fonction pour récupérer la température d'une ville
def get_temperature(latitude, longitude):
    url = f"https://api.openweathermap.org/data/3.0/onecall?lat={latitude}&lon={longitude}&appid={api_key}"
    response = requests.get(url)
    data = response.json()
    if "current" in data:
        temperature_celsius = data["current"]["temp"] - 273.15  # Convertir de Kelvin en Celsius
        return temperature_celsius
    else:
        return None

# Parcourir les lignes du DataFrame
for index, row in df.iterrows():
    ville = row['Ville']
    latitude = row['Latitude']
    longitude = row['Longitude']
    temperature = get_temperature(latitude, longitude)
    if temperature is not None:
        print(f"Ville: {ville}")
        print(f"Température : {temperature:.1f}°")
    else:
        print(f"Aucune donnée météorologique disponible pour {ville}")

Ville: Mont Saint Michel
Température : 16.8°
Ville: St Malo
Température : -1.2°
Ville: Bayeux
Température : 16.1°
Ville: Le Havre
Température : 14.5°
Ville: Rouen
Température : 16.1°
Ville: Paris
Température : 15.0°
Ville: Amiens
Température : 14.6°
Ville: Lille
Température : 14.9°
Ville: Strasbourg
Température : 14.0°
Ville: Chateau du Haut Koenigsbourg
Température : 11.4°
Ville: Colmar
Température : 15.5°
Ville: Eguisheim
Température : 15.1°
Ville: Besancon
Température : 15.6°
Ville: Dijon
Température : 12.7°
Ville: Annecy
Température : 16.1°
Ville: Grenoble
Température : 17.8°
Ville: Lyon
Température : 18.8°
Ville: Gorges du Verdon
Température : 17.5°
Ville: Bormes les Mimosas
Température : 18.6°
Ville: Cassis
Température : 20.3°
Ville: Marseille
Température : 21.3°
Ville: Aix en Provence
Température : 20.5°
Ville: Avignon
Température : 20.3°
Ville: Uzes
Température : 18.9°
Ville: Nimes
Température : 18.5°
Ville: Aigues Mortes
Température : 18.0°
Ville: Saintes Maries de la mer
Temp